# 1. 실험 준비

## 1-1. 라이브러리 선언

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from matplotlib.colors import LinearSegmentedColormap

from util.util import data_split, get_label, ECG_metrics, PPG_metrics, process_subject_PTT, process_biosignals_rr_si, process_biosignals_gsr, save_joined_features, load_subject_data, process_labels, get_corr

## 1-2. Subject 및 전역 변수 선언

In [2]:
subject_list =[
 '1_1_011_V2',
 '1_1_015_V2',
 '1_1_025_V1',
 '1_1_027_V1',
 '1_1_028_V1',
 '1_1_029_V1',
 '1_1_034_V1',
 '1_1_035_V1',
 '1_1_036_V1',
 ]
# 예시 Subject
subject = '1_1_011_V2'

DATA_FOLDER = r'..\features\joined\\'
OUTPUT_FOLDER = r'..\features\results\\'
label_path = r'..\features\label'

# 2. Raw 데이터 준비
## 2-1. 데이터 분할 (Data Split)

In [5]:
raw_data_dir = '../data/raw_data/'
split_save_dir = '../data/split_data/'

for subject_name in subject_list:
    data_split(raw_data_dir+subject_name, split_save_dir+subject_name)

data_split_result = pd.read_csv('../data/split_data/'+subject+'/ECG/high/high1.csv')
data_split_result

../data/raw_data/1_1_011_V2 데이터 Split 처리 시작
../data/split_data/1_1_011_V2폴더에 Split데이터를 저장했습니다.

../data/raw_data/1_1_015_V2 데이터 Split 처리 시작
../data/split_data/1_1_015_V2폴더에 Split데이터를 저장했습니다.

../data/raw_data/1_1_025_V1 데이터 Split 처리 시작
../data/split_data/1_1_025_V1폴더에 Split데이터를 저장했습니다.

../data/raw_data/1_1_027_V1 데이터 Split 처리 시작
../data/split_data/1_1_027_V1폴더에 Split데이터를 저장했습니다.

../data/raw_data/1_1_028_V1 데이터 Split 처리 시작
../data/split_data/1_1_028_V1폴더에 Split데이터를 저장했습니다.

../data/raw_data/1_1_029_V1 데이터 Split 처리 시작
../data/split_data/1_1_029_V1폴더에 Split데이터를 저장했습니다.

../data/raw_data/1_1_034_V1 데이터 Split 처리 시작
../data/split_data/1_1_034_V1폴더에 Split데이터를 저장했습니다.

../data/raw_data/1_1_035_V1 데이터 Split 처리 시작
../data/split_data/1_1_035_V1폴더에 Split데이터를 저장했습니다.

../data/raw_data/1_1_036_V1 데이터 Split 처리 시작
../data/split_data/1_1_036_V1폴더에 Split데이터를 저장했습니다.



,Shimmer_820D_Timestamp_Unix_CAL,Shimmer_820D_ECG_EMG_Status1_CAL,Shimmer_820D_ECG_EMG_Status2_CAL,Shimmer_820D_ECG_LA-RA_24BIT_CAL,Shimmer_820D_ECG_LL-LA_24BIT_CAL,Shimmer_820D_ECG_LL-RA_24BIT_CAL,Shimmer_820D_ECG_Vx-RL_24BIT_CAL,Unnamed: 7
0,1.757987e+12,128.0,128.0,0.750786,-2.822913,-2.072127,-12.091840,NaN
1,1.757987e+12,128.0,128.0,0.745954,-2.824571,-2.078617,-12.066670,NaN
2,1.757987e+12,128.0,128.0,0.751003,-2.806613,-2.055611,-12.115280,NaN
3,1.757987e+12,128.0,128.0,0.743141,-2.794280,-2.051139,-12.092057,NaN
4,1.757987e+12,128.0,128.0,0.745016,-2.777692,-2.032676,-12.106048,NaN
...,...,...,...,...,...,...,...,...
25710,1.757987e+12,128.0,128.0,1.500995,-1.944471,-0.443476,-11.867542,NaN
25711,1.757987e+12,128.0,128.0,1.502582,-1.959977,-0.457395,-11.846338,NaN
25712,1.757987e+12,128.0,128.0,1.518377,-1.949087,-0.430710,-11.874249,NaN
25713,1.757987e+12,128.0,128.0,1.522776,-1.924926,-0.402150,-11.900934,NaN


## 2-2. Data Label Generation

In [6]:
base_dir = r'../data/raw_data/'
sam_result_path = r'../data/SAM/'
save_dir = '../features/label/'
Label_result = get_label(base_dir=base_dir+subject, sam_result_path=sam_result_path, subject_path=subject, output_dir=save_dir)
Label_result

../data/raw_data/1_1_011_V2 Label 정보 처리 시작
../features/label/1_1_011_V2.csv에 Label 정보를 저장했습니다.



,File,label-3.5_more,label-3.5_over,label-avg,Q1,Q2
0,low1.csv,0,0,0.0,0,0
1,low2.csv,0,0,0.0,0,0
2,low3.csv,0,0,0.0,0,0
3,low4.csv,0,0,0.0,0,0
4,low5.csv,0,0,0.0,0,0
5,low6.csv,0,0,0.0,0,0
6,mid1.csv,0,0,0.0,0,0
7,mid2.csv,0,0,1.5,3,0
8,mid3.csv,0,0,0.0,0,0
9,mid4.csv,0,0,0.0,0,0


# 3. 생체신호 특징 추출
## 3-1. ECG, PPG HR_HRV
### 3-1-1. ECG HR_HRV

In [7]:
split_data_dir = r"..\data\split_data"
save_path = "..\\features\\HR_HRV\\ECG\\"+subject
# Save
ECG_HR_HRV_result = ECG_metrics(os.path.join(split_data_dir, subject)+"\\ECG", save_path=save_path)
ECG_HR_HRV_result

..\data\split_data\1_1_011_V2\ECG ECG Signal - HR, HRV 분석 시작
..\features\HR_HRV\ECG\1_1_011_V2\1_1_011_V2.csv에 ECG Signal - HR, HRV 분석 결과를 저장했습니다.


,File,channel,HR,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,end1.csv,LA_RA,78.350726,765.755208,29.129826,NaN,NaN,NaN,NaN,NaN,...,1.491655,5.081631,1.368162,1.275449,0.807091,1.337719,1.760663,1.862438,3.145242,1.245764
1,end1.csv,LL_LA,78.352829,765.755208,29.184634,NaN,NaN,NaN,NaN,NaN,...,2.484907,5.211957,1.354527,1.257609,0.983438,1.361536,1.778267,1.866197,3.145242,1.162713
2,end1.csv,LL_RA,78.356089,765.729167,29.164420,NaN,NaN,NaN,NaN,NaN,...,1.763589,5.223698,1.344070,1.309441,0.856398,1.278211,1.784201,1.867414,3.094883,1.245764
3,end1.csv,Vx_RL,78.350560,765.755208,29.136904,NaN,NaN,NaN,NaN,NaN,...,2.427748,5.197031,1.354882,0.953931,1.217621,1.500527,1.842993,1.863170,3.075252,1.245764
4,high1.csv,LA_RA,74.849269,801.869877,31.661364,NaN,NaN,NaN,NaN,NaN,...,1.658228,4.938453,1.310519,1.362404,0.927903,1.178831,1.396763,1.838432,3.496750,1.069477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,mid5.csv,Vx_RL,73.241312,819.295247,27.488780,NaN,NaN,NaN,NaN,NaN,...,inf,4.761842,1.496542,NaN,0.000000,0.000000,2.491066,1.940085,4.592174,1.163534
76,start1.csv,LA_RA,79.730734,752.621299,59.158350,NaN,NaN,NaN,NaN,NaN,...,1.172720,5.691814,0.868742,0.871618,1.019680,1.343463,1.715302,1.478053,1.826775,0.575467
77,start1.csv,LL_LA,79.725996,752.646998,59.213480,NaN,NaN,NaN,NaN,NaN,...,1.247032,5.612866,0.870433,0.624949,0.968962,1.261006,1.672951,1.480150,1.823616,0.575467
78,start1.csv,LL_RA,79.726315,752.646998,59.228939,NaN,NaN,NaN,NaN,NaN,...,1.187843,5.675430,0.868395,0.985380,0.974733,1.220224,1.736408,1.483196,1.824611,0.575467


## 3-1-2. PPG HR_HRV

In [8]:
split_data_dir = r"..\data\split_data"
save_path = "..\\features\\HR_HRV\\PPG\\"+subject
# Save
PPG_HR_HRV_result = PPG_metrics(os.path.join(split_data_dir, subject)+"\\PPG", save_path=save_path)
PPG_HR_HRV_result

..\data\split_data\1_1_011_V2\PPG PPG Signal - HR, HRV 분석 시작
..\features\HR_HRV\PPG\1_1_011_V2\1_1_011_V2.csv에 PPG Signal - HR, HRV 분석 결과를 저장했습니다.


,File,channel,HR,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,end1.csv,ppg,78.348128,766.145833,30.219041,NaN,NaN,NaN,NaN,NaN,...,1.746909,2.581590,1.409175,1.205974,0.995721,1.100123,0.871467,1.828831,3.258405,1.079662
1,high1.csv,ppg,74.778557,802.382172,32.544612,NaN,NaN,NaN,NaN,NaN,...,1.536235,2.603172,1.262478,1.151919,0.893195,0.882681,0.823186,1.786617,2.623491,1.069477
2,high2.csv,ppg,75.061770,799.457097,33.607610,NaN,NaN,NaN,NaN,NaN,...,1.276293,2.738377,1.451623,0.593726,0.871824,0.905863,0.814910,1.978206,2.519035,0.897352
3,high3.csv,ppg,72.514633,827.256944,31.204344,NaN,NaN,NaN,NaN,NaN,...,1.417066,2.571013,1.567166,0.000000,0.866060,0.874127,0.629120,2.013709,3.026159,0.976329
4,high4.csv,ppg,73.943779,811.567164,34.217052,NaN,NaN,NaN,NaN,NaN,...,2.014903,2.749999,1.484751,0.951666,1.368390,1.350196,1.051886,1.866853,3.224590,1.177002
5,high5.csv,ppg,71.299928,841.557018,37.329130,NaN,NaN,NaN,NaN,NaN,...,1.148623,2.830006,1.191028,0.000000,0.955598,1.318366,0.844857,1.716691,2.756331,0.716320
6,high6.csv,ppg,75.481918,795.200893,22.116327,NaN,NaN,NaN,NaN,NaN,...,1.763589,2.178657,1.991092,0.843257,0.770779,0.765718,0.372395,2.031563,4.410982,1.283885
7,high7.csv,ppg,74.436819,806.189904,38.987307,NaN,NaN,NaN,NaN,NaN,...,1.540445,2.961880,1.528761,0.834363,1.153326,1.711985,1.068179,1.783813,3.544902,0.833866
8,low1.csv,ppg,75.823003,791.638963,26.029619,NaN,NaN,NaN,NaN,NaN,...,1.763589,2.375179,1.589706,0.715550,0.798672,0.802627,0.509914,1.815162,3.846226,1.181827
9,low2.csv,ppg,73.767095,813.419118,29.959570,NaN,NaN,NaN,NaN,NaN,...,1.704748,2.583877,1.532479,0.000000,0.912665,1.103200,0.882456,1.945821,3.340232,1.223464


## 3-2. PTT Calculate

In [9]:
save_dir = r"../features/PTT"
split_data_dir = r"../data/split_data"

PTT_result = process_subject_PTT(os.path.join(split_data_dir, subject), subject, save_path=save_dir)
PTT_result

../data/split_data\1_1_011_V2 PTT 분석 시작
../features/PTT\1_1_011_V2.csv에 PTT 계산 결과를 저장했습니다.


,Subject,File,LA-RA_PTT_avg,LA-RA_PTT_std,LL-LA_PTT_avg,LL-LA_PTT_std,LL-RA_PTT_avg,LL-RA_PTT_std,Vx-RL_PTT_avg,Vx-RL_PTT_std
0,1_1_011_V2,start1.csv,340.325234,4.878096,319.018390,4.546275,325.258260,4.557075,351.739584,4.660003
1,1_1_011_V2,low1.csv,342.447458,2.743107,321.410563,2.355311,327.636333,2.307750,353.962625,2.621523
2,1_1_011_V2,low2.csv,346.392216,2.560381,325.213804,2.267704,331.265078,2.190498,358.302216,2.425316
3,1_1_011_V2,low3.csv,344.530933,8.825422,323.176378,8.672358,329.383422,8.667981,356.888795,9.097955
4,1_1_011_V2,low4.csv,346.201532,6.674837,325.174021,6.691981,331.199787,6.543173,358.834426,6.628960
5,1_1_011_V2,low5.csv,339.212169,3.197527,318.028446,3.150460,324.638831,2.896865,351.652154,3.364131
6,1_1_011_V2,low6.csv,340.891293,3.860763,319.359268,3.979781,325.838024,3.831866,352.991073,3.977542
7,1_1_011_V2,mid1.csv,342.569604,3.232687,320.881625,2.731930,327.392167,2.864705,354.898542,3.008923
8,1_1_011_V2,mid2.csv,339.094550,19.991578,317.414883,20.154398,323.892667,20.235313,348.404383,30.908540
9,1_1_011_V2,mid3.csv,348.108681,4.704282,326.701614,4.573676,333.063143,4.413494,360.676710,4.768780


## 3-3. Stress Index(SI) & Respiratory Rate(RR) Calculate

In [10]:
save_dir = r"../features/"
split_data_dir = r"../data/split_data/"
_ = process_biosignals_rr_si(split_data_dir+subject+r'\\ECG', 'ECG', save_path=save_dir)
_ = process_biosignals_rr_si(split_data_dir+subject+r'\\PPG', 'PPG', save_path=save_dir)

RR_ECG_result = pd.read_csv(r'..\features\RR\ECG\\'+subject+'\\'+subject+r'.csv')
RR_PPG_result = pd.read_csv(r'..\features\RR\PPG\\'+subject+'\\'+subject+r'.csv')

SI_ECG_result = pd.read_csv(r'..\features\SI\ECG\\'+subject+'\\'+subject+r'.csv')
SI_PPG_result = pd.read_csv(r'..\features\SI\PPG\\'+subject+'\\'+subject+r'.csv')

../data/split_data/1_1_011_V2\\ECG ECG Signal - SI, RR 분석 시작
../features/RR\ECG\1_1_011_V2\1_1_011_V2.csv에 ECG Signal - RR 분석 결과를 저장했습니다.
../features/RR\ECG\1_1_011_V2\1_1_011_V2.csv에 ECG Signal - SI 분석 결과를 저장했습니다.

../data/split_data/1_1_011_V2\\PPG PPG Signal - SI, RR 분석 시작
../features/RR\PPG\1_1_011_V2\1_1_011_V2.csv에 PPG Signal - RR 분석 결과를 저장했습니다.
../features/RR\PPG\1_1_011_V2\1_1_011_V2.csv에 PPG Signal - SI 분석 결과를 저장했습니다.



In [11]:
RR_ECG_result

,File,LA_RA_rr,LL_LA_rr,LL_RA_rr,Vx_RL_rr
0,end1.csv,21.801401,21.801401,21.757879,21.801401
1,high1.csv,23.977489,23.977489,23.977489,23.977489
2,high2.csv,22.410705,22.367184,22.367184,22.410705
3,high3.csv,21.540270,21.583792,21.583792,21.018009
4,high4.csv,23.672836,23.672836,23.716358,23.672836
5,high5.csv,23.977489,23.977489,23.977489,23.977489
6,high6.csv,12.226613,12.226613,12.226613,12.226613
7,high7.csv,23.977489,23.977489,23.977489,23.977489
8,low1.csv,22.367184,22.367184,22.323662,22.367184
9,low2.csv,17.753877,17.710355,17.710355,17.710355


In [12]:
RR_PPG_result

,File,ppg_rr
0,end1.csv,21.670835
1,high1.csv,23.977489
2,high2.csv,22.410705
3,high3.csv,21.105053
4,high4.csv,23.759880
5,high5.csv,23.977489
6,high6.csv,21.931966
7,high7.csv,23.977489
8,low1.csv,22.367184
9,low2.csv,17.840920


In [13]:
SI_ECG_result

,File,LA_RA_si,LL_LA_si,LL_RA_si,Vx_RL_si
0,end1.csv,6.049120,4.951542,5.599995,5.453788
1,high1.csv,7.041724,6.278097,6.371342,6.371342
2,high2.csv,5.953154,5.960647,6.433375,6.485050
3,high3.csv,4.328427,4.263869,4.303813,4.348189
4,high4.csv,5.472075,5.402364,4.837811,4.900237
5,high5.csv,4.962387,4.766804,4.653969,4.687572
6,high6.csv,5.624757,5.737546,4.895061,4.871183
7,high7.csv,4.958591,5.008302,3.899336,5.578415
8,low1.csv,8.736642,7.210610,6.244571,8.061707
9,low2.csv,6.629277,5.992449,5.992449,5.886282


In [14]:
SI_PPG_result

,File,ppg_si
0,end1.csv,12.395060
1,high1.csv,12.075019
2,high2.csv,10.539353
3,high3.csv,11.639370
4,high4.csv,10.361363
5,high5.csv,10.083599
6,high6.csv,15.111202
7,high7.csv,8.533333
8,low1.csv,12.935454
9,low2.csv,11.062677


## 3-4. GSR Feature

In [3]:
save_dir = r"../features/"
split_data_dir = r"../data/split_data/"

GSR_result = process_biosignals_gsr(split_data_dir+ subject+r'\\PPG', save_path=save_dir)
GSR_result

../data/split_data/1_1_011_V2\\PPG GSR Signal - 분석 시작
../features/EDA\1_1_011_V2\1_1_011_V2.csv에 GSR Signal - 분석 결과를 저장했습니다.



,File,SCR_Peaks_N,SCR_Peaks_Amplitude_Mean,EDA_Tonic_SD,EDA_Sympathetic,EDA_SympatheticN,EDA_Autocorrelation
0,end1.csv,36.0,0.015264,0.989061,NaN,NaN,0.609320
1,high1.csv,6.0,0.531363,0.941305,NaN,NaN,NaN
2,high2.csv,6.0,0.894028,0.729815,NaN,NaN,NaN
3,high3.csv,4.0,0.967398,0.409341,NaN,NaN,NaN
4,high4.csv,6.0,0.082469,0.995638,NaN,NaN,NaN
5,high5.csv,5.0,0.356453,0.984535,NaN,NaN,NaN
6,high6.csv,5.0,0.511750,0.958097,NaN,NaN,NaN
7,high7.csv,3.0,0.593255,0.924285,NaN,NaN,NaN
8,low1.csv,7.0,0.455644,0.877900,NaN,NaN,NaN
9,low2.csv,8.0,0.214763,1.008107,NaN,NaN,NaN


# 4. 분석 데이터 준비
## 4-1. Feature Integration

In [16]:
try:
    save_joined_features(subject)
    joined_df, label_df = load_subject_data(fname = subject+'.csv')
except Exception as e:
    print(e)
joined_df

1_1_011_V2 추출한 특징 파일 병합 시작
[Errno 2] No such file or directory: '../features/EDA/1_1_011_V2/1_1_011_V2.csv'


NameError: name 'joined_df' is not defined

## 4-2. Label 처리

In [ ]:
processed_label_df = process_labels(label_df = label_df)
processed_label_df

## 4-3. 데이터 요약 및 통계 계산
---
### calculate_summary_metrics
##### - 통합 CSV 파일을 읽어 정제 후, 키워드 기준으로 그룹화하여 통계 지표(감소율, 유의미성)를 계산.
- Args:
    - joint_dir (str): 분석할 데이터가 포함된 단일 CSV 파일의 전체 경로.
- Returns:
    - summary_df (pd.DataFrame): 계산된 모든 요약 통계 지표를 담고 있는 데이터프레임.
        - Index: 계산된 통계 지표의 이름.
        - intervene_val: 'intervene' 키워드를 포함하는 데이터의 평균값.
        - craving_max: 특정 키워드('intervene', 'control', 'end')를 제외한 데이터의 최대값.
        - decline_rate: intervene_val을 craving_max로 나눈 감소율.
        - is_significant: decline_rate이 0.9 이하인지 여부를 나타내는 boolean 값.
        - Columns: 데이터 정제 후 최종 피처(feature)들의 이름.
        - Values: 각 피처에 대해 계산된 통계 지표 값.

### Function analyze_and_save_subjects
##### - 지정된 폴더의 모든 CSV 파일을 개별 분석하고, 각 결과를 별도의 CSV 파일로 저장하는 함수.
- Args:
    - input_folder (str): 원본 데이터 CSV 파일들이 있는 폴더 경로.
    - output_folder (str): 결과 CSV 파일을 저장할 폴더 경로.
    - subject_list (list): 사용할  subject의 리스트.

In [ ]:
def calculate_summary_metrics(joint_dir):
    '''
    통합 CSV 파일을 읽어 정제 후, 키워드 기준으로 그룹화하여 통계 지표(감소율, 유의미성)를 계산.

    Args:
        joint_dir (str): 분석할 데이터가 포함된 단일 CSV 파일의 전체 경로.

    Returns:
        summary_df (pd.DataFrame): 계산된 모든 요약 통계 지표를 담고 있는 데이터프레임.
            Index: 계산된 통계 지표의 이름.
            intervene_val: 'intervene' 키워드를 포함하는 데이터의 평균값.
            craving_max: 특정 키워드('intervene', 'control', 'end')를 제외한 데이터의 최대값.
            decline_rate: intervene_val을 craving_max로 나눈 감소율.
            is_significant: decline_rate이 0.9 이하인지 여부를 나타내는 boolean 값.
            Columns: 데이터 정제 후 최종 피처(feature)들의 이름.
            Values: 각 피처에 대해 계산된 통계 지표 값.
    '''
    df = pd.read_csv(joint_dir)
    # 데이터프레임 NaN값 들어있는 열 제거
    df = df.dropna (axis=1)

    # 데이터프레임 음수값 들어있는 열 제거
    numeric_cols = df.select_dtypes(include=np.number).columns
    cols_with_negatives = [col for col in numeric_cols if (df[col] < 0).any()]

    # 데이터프레임 0, inf, -inf값 들어있는 열 제거
    values_to_remove = [0, np.inf, -np.inf]
    cols_with_unwanted = df[numeric_cols].isin(values_to_remove).any()
    cols_to_drop_unwanted = cols_with_unwanted[cols_with_unwanted].index

    all_cols_to_drop = list(set(cols_with_negatives) | set(cols_to_drop_unwanted))
    df_filtered = df.drop(columns=all_cols_to_drop)

    feature_cols = df_filtered.select_dtypes(include=np.number).columns

    # intervene1.csv 가져오기
    intervene_mask = df_filtered['File'].str.contains('intervene', case=False, na=False)
    intervene_means = df_filtered[intervene_mask][feature_cols].mean()

    # end, control, intervene1 제외 열에서(Feature 마다) 가장 높은 값 가져오기
    exclude_keywords = 'start|intervene|control|end'
    exclude_mask = df_filtered['File'].str.contains(exclude_keywords, case=False, na=False)
    craving_maxes = df_filtered[~exclude_mask][feature_cols].max()

    summary_df = pd.DataFrame({
        'intervene_val': intervene_means,
        'craving_max': craving_maxes
    }).T

    decline_rate = (summary_df.loc['intervene_val'] / summary_df.loc['craving_max'].replace(0, np.nan))
    summary_df.loc['decline_rate'] = decline_rate
    summary_df.loc['is_significant'] = summary_df.loc['decline_rate'] <= 0.9

    return summary_df

def analyze_and_save_subjects(input_folder, output_folder, subject_list):
    '''
    지정된 폴더의 모든 CSV 파일을 개별 분석하고, 각 결과를 별도의 CSV 파일로 저장하는 함수.

    Args:
        input_folder (str): 원본 데이터 CSV 파일들이 있는 폴더 경로.
        output_folder (str): 결과 CSV 파일을 저장할 폴더 경로.
        subject_list (list): 사용할  subject의 리스트.
    '''

    os.makedirs(output_folder, exist_ok=True)

    csv_files = glob.glob(os.path.join(input_folder, '*.csv'))

    if not csv_files:
        print(f"경고: '{input_folder}' 폴더에서 CSV 파일을 찾을 수 없습니다.")
        return

    for file_path in csv_files:
        subject_id = os.path.basename(file_path)
        print(f"Analyzing: {subject_id}...")
        try:
            subject_results = calculate_summary_metrics(file_path)
            subject_df = pd.DataFrame(subject_results)

            output_filename = f"result_{subject_id}"
            output_path = os.path.join(output_folder, output_filename)
            subject_df.to_csv(output_path)

        except Exception as e:
            print(f"'{subject_id}' 파일 처리 중 오류 발생: {e}")

    print(f"\n모든 분석이 완료되었습니다. 결과는 '{output_folder}' 폴더에 저장되었습니다.")

In [ ]:
analyze_and_save_subjects(DATA_FOLDER, OUTPUT_FOLDER, subject_list)
analyze_result = pd.read_csv(OUTPUT_FOLDER+'/result_'+subject+'.csv')
analyze_result

### Function combine_decline_rates
##### - 지정된 폴더(directory) 내의 모든 CSV 파일을 읽어, 각 파일에서 'decline_rate' 행의 데이터를 추출. 데이터들을 하나의 데이터프레임으로 병합한 후, 모든 파일에 공통으로 존재하는 피처(feature)들만 남겨 최종 결과를 반환하는 함수.
- Args
    - directory_path (str): 요약 정보가 담긴 CSV 파일들이 저장되어 있는 폴더의 경로.
- Returns
    - pd.DataFrame: 아래 두 가지 경우 중 하나에 해당하는 데이터프레임을 반환.
        - Index: 원본 CSV 파일의 이름 (예: 'result_1_01_011_V1.csv').
        - Columns: 모든 CSV 파일에서 공통으로 발견된 피처(feature)들의 이름.
        - Values: 각 파일의 피처별 decline_rate 값.

In [ ]:
def combine_decline_rates(directory_path):
    '''
    지정된 폴더(directory) 내의 모든 CSV 파일을 읽어, 각 파일에서 'decline_rate' 행의 데이터를 추출. 데이터들을 하나의 데이터프레임으로 병합한 후, 모든 파일에 공통으로 존재하는 피처(feature)들만 남겨 최종 결과를 반환하는 함수.
    Args
        directory_path (str): 요약 정보가 담긴 CSV 파일들이 저장되어 있는 폴더의 경로.
    Returns
        pd.DataFrame: 아래 두 가지 경우 중 하나에 해당하는 데이터프레임을 반환.
            Index: 원본 CSV 파일의 이름 (예: 'result_1_01_011_V1.csv').
            Columns: 모든 CSV 파일에서 공통으로 발견된 피처(feature)들의 이름.
            Values: 각 파일의 피처별 decline_rate 값.
    '''
    try:
        csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("지정된 경로에 CSV 파일이 없습니다.")
        print(f"총 {len(csv_files)}개의 요약 CSV 파일을 대상으로 작업을 시작합니다...")

    except FileNotFoundError as e:
        print(e)
        return

    all_decline_rates = []
    for filename in csv_files:
        filepath = os.path.join(directory_path, filename)
        try:
            summary_df = pd.read_csv(filepath, index_col=0)

            if 'decline_rate' in summary_df.index:
                decline_rate_series = summary_df.loc['decline_rate']
                decline_rate_series.name = filename
                all_decline_rates.append(decline_rate_series)
            else:
                print(f"  - 경고: '{filename}' 파일에 'decline_rate' 행이 없습니다.")

        except Exception as e:
            print(f"  - 오류: '{filename}' 파일을 처리하는 중 문제가 발생했습니다: {e}")

    if all_decline_rates:
        final_df = pd.concat(all_decline_rates, axis=1).T
        final_common_df = final_df.dropna(axis=1)

        print("\n--- 최종 결과: 공통 피쳐의 decline_rate ---")
        return final_common_df
    else:
        print("\n처리할 데이터가 없습니다.")
        return pd.DataFrame()

In [ ]:
decline_rates_series = combine_decline_rates(OUTPUT_FOLDER).max()
significant_features_dict = decline_rates_series[decline_rates_series <= 0.9].to_dict()
print(len(significant_features_dict))

# 5. 바이오마커 정상화 수치 계산
## 5-1. 상관계수 계산

In [ ]:
corr_df = get_corr(joined_dir=DATA_FOLDER)
corr_df

### Function generate_corr_sign_dict
##### - mean corr 추출하여 DataFrame의 특정 열에 있는 상관계수 값의 부호를 분류하는 딕셔너리를 생성하는 함수.
- Args:
    - corr_df (pd.DataFrame): 상관계수 데이터가 포함된 DataFrame.
- Returns:
    - dict: 피처 이름을 키(key)로, 'pos' 또는 'neg'를 값(value)으로 갖는 딕셔너리.
        - Key: CSV 파일에서 공통으로 발견된 피처(feature)들의 이름.
        - Values: pos or neg.
            - 값이 양수이면 'pos'로 매핑함
            - 값이 음수이면 'neg'로 매핑함
            - NaN 또는 0인 값은 무시함


In [ ]:
def generate_corr_sign_dict(corr_df):
    '''
    mean corr 추출하여 DataFrame의 특정 열에 있는 상관계수 값의 부호를 분류하는 딕셔너리를 생성하는 함수.
    Args:
        corr_df (pd.DataFrame): 상관계수 데이터가 포함된 DataFrame.
    Returns:
        dict: 피처 이름을 키(key)로, 'pos' 또는 'neg'를 값(value)으로 갖는 딕셔너리.
            Key: CSV 파일에서 공통으로 발견된 피처(feature)들의 이름.
            Values: pos or neg.
                값이 양수이면 'pos'로 매핑함
                값이 음수이면 'neg'로 매핑함
                NaN 또는 0인 값은 무시함
    '''
    corr_sign_dict = {
        feature: 'pos' if value > 0 else 'neg'
        for feature, value in corr_df.dropna().items() if value != 0
    }
    return corr_sign_dict

In [ ]:
all_sign_dict = generate_corr_sign_dict(corr_df['mean_corr'])
all_sign_dict

### Function get_intervene
##### - 두 개의 딕셔너리를 입력받아, 특정 조건('pos' 사인)을 만족하는 피처들만 선택하고 값을 계산하여 새로운 데이터프레임으로 반환하는 함수.
- Args
    - all_sign_dict (dict): 모든 피처의 사인(sign) 정보가 담긴 딕셔너리.
        - Key: 피처 이름 (str)
        - Value: 해당 피처의 사인 (str, 예: 'pos', 'neg')
    - significant_features_dict (dict): 유의미한 피처들의 비율(ratio) 값이 담긴 딕셔너리.
        - Key: 피처 이름 (str)
        - Value: 해당 피처의 비율 값 (float 또는 int)
- Returns
    - extract_df (pd.DataFrame): 아래 조건에 따라 필터링되고 계산된 결과를 담은 데이터프레임.
        -  all_sign_dict에서 값이 'pos'이고, significant_features_dict에도 존재하는 피처들만 포함.
            - Index: 선택된 피처들의 이름.
            - Column: 'intervene_Ratio'라는 단일 컬럼.
            - Value: 입력된 significant_features_dict의 원래 값에서 **1 - 원래 값**으로 계산된 새로운 값 (정상화 비율).

In [ ]:
def get_intervene(all_sign_dict, significant_features_dict):
    '''
    두 개의 딕셔너리를 입력받아, 특정 조건('pos' 사인)을 만족하는 피처들만 선택하고 값을 계산하여 새로운 데이터프레임으로 반환하는 함수.

    Args
        all_sign_dict (dict): 모든 피처의 사인(sign) 정보가 담긴 딕셔너리.
            Key: 피처 이름 (str)
            Value: 해당 피처의 사인 (str, 예: 'pos', 'neg')
        significant_features_dict (dict): 유의미한 피처들의 비율(ratio) 값이 담긴 딕셔너리.
            Key: 피처 이름 (str)
            Value: 해당 피처의 비율 값 (float 또는 int)

    Returns
        extract_df (pd.DataFrame): 아래 조건에 따라 필터링되고 계산된 결과를 담은 데이터프레임.
            all_sign_dict에서 값이 'pos'이고, significant_features_dict에도 존재하는 피처들만 포함.
            Index: 선택된 피처들의 이름.
            Column: 'intervene_Ratio'라는 단일 컬럼.
            Value: 입력된 significant_features_dict의 원래 값에서 **1 - 원래 값**으로 계산된 새로운 값 (정상화 비율).
    '''
    pos_sign_dict = {
        key: value
        for key, value in all_sign_dict.items()
        if value == 'pos'
    }
    pos_dict_label = list(pos_sign_dict.keys())

    significant_features_df = pd.DataFrame(significant_features_dict, index=['intervene_Ratio']).T
    significant_features_df['intervene_Ratio'] = 1 - significant_features_df['intervene_Ratio']
    df_keys = significant_features_df.index.tolist()
    valid_keys = [key for key in pos_dict_label if key in df_keys]
    extract_df = significant_features_df.loc[valid_keys]
    print(len(valid_keys))
    return extract_df

In [ ]:
intervene_df = get_intervene(all_sign_dict, significant_features_dict)
intervene_df

## 5-2. 결과 시각화
---
#### Function plot_intervene_features
##### - 데이터프레임에 포함된 피처(feature)별 'Intervention Ratio'를 정렬하여 Seaborn 라이브러리의 히트맵(heatmap)으로 시각화하고, 그 결과를 이미지 파일로 저장하는 함수.
- Args
    - df (pd.DataFrame): 시각화의 기반이 될 데이터프레임입니다. 아래의 구조를 반드시 만족.
        - 인덱스 (Index): 각 피처(feature)의 고유한 이름 (예: 'SCR_Peaks_Amplitude_Mean')
        - 컬럼 (Column): 'intervene_Ratio'라는 이름의 숫자형 열을 반드시 포함.
    - filename (str, optional): 저장될 이미지 파일의 경로와 이름. 별도로 지정하지 않으면 기본값인 'feature_heatmap.png'로 저장.
    - title (str, optional): 생성될 히트맵의 상단에 표시될 제목. 별도로 지정하지 않으면 기본값인 'Biomarker'가 사용.

In [ ]:
def plot_intervene_features(df, filename='feature_heatmap.png', title='Biomarker'):
    '''
    ##### - 데이터프레임에 포함된 피처(feature)별 'Intervention Ratio'를 정렬하여 Seaborn 라이브러리의 히트맵(heatmap)으로 시각화하고, 그 결과를 이미지 파일로 저장하는 함수.
    Args
        df (pd.DataFrame): 시각화의 기반이 될 데이터프레임입니다. 아래의 구조를 반드시 만족.
            인덱스 (Index): 각 피처(feature)의 고유한 이름 (예: 'SCR_Peaks_Amplitude_Mean')
            컬럼 (Column): 'intervene_Ratio'라는 이름의 숫자형 열을 반드시 포함.
        filename (str, optional): 저장될 이미지 파일의 경로와 이름. 별도로 지정하지 않으면 기본값인 'feature_heatmap.png'로 저장.
        title (str, optional): 생성될 히트맵의 상단에 표시될 제목. 별도로 지정하지 않으면 기본값인 'Biomarker'가 사용.
    '''

    if df.empty:
        print("⚠️ Warning: Input DataFrame is empty. Plotting is skipped.")
        return

    # 1. 데이터 준비 (이전과 동일: 정렬 및 행/열 전환)
    df_copy = df.copy()
    df_sorted = df_copy.sort_values(by='intervene_Ratio', ascending=False)
    df_transposed = df_sorted.T # <-- Transpose를 먼저 실행
    df_transposed.rename(index={'intervene_Ratio': 'Intervention Ratio'}, inplace=True) # <-- Transpose된 DF의 인덱스를 변경

    # 2. Figure 크기 동적 계산
    num_features = len(df_transposed.columns)
    fig_width = max(10, num_features * 0.8) # 최소 너비 12인치 확보
    fig_height = 4 # 높이는 상대적으로 고정

    plt.figure(figsize=(fig_width, fig_height))

    # 3. 컬러맵 설정
    cmap = LinearSegmentedColormap.from_list("white_to_yellow", ["#ffffff", "#ffcc00"])

    # --- 여기가 핵심: Seaborn Heatmap 사용 ---
    ax = sns.heatmap(
        data=df_transposed,
        annot=True,          # 셀 안에 숫자(값) 표시
        fmt=".4f",           # 숫자를 소수점 4자리까지 표시
        cmap=cmap,           # 위에서 정의한 커스텀 컬러맵 사용
        # linewidths=.5,       # 셀 사이에 가는 실선 추가
        # cbar_kws={'label': 'Intervention Ratio'} # 컬러바 레이블 설정
    )
    # ----------------------------------------

    # 4. 플롯 스타일링
    ax.set_title(title, fontsize=16, weight='bold')

    # x축 레이블 (피처 이름)을 45도 회전하여 가독성 확보
    plt.xticks(rotation=45, ha='right')

    # y축 레이블 ('intervene_Ratio')은 수평으로 표시
    plt.yticks(rotation=90, weight='bold')

    # 레이블이 잘리지 않도록 레이아웃 자동 조정
    plt.tight_layout()

    # 5. 이미지 저장 및 출력
    plt.savefig(filename, dpi=600)
    plt.show()

    print(f"✅ 이미지 '{filename}' 저장 및 출력 성공!")

In [ ]:
plot_intervene_features(intervene_df)